# **CIANNA ImageNET example script**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/CIANNA/blob/CIANNA/examples/ImageNET/imagenet_pred_notebook.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [ ]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the last cell to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [ ]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [ ]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64

#### CIANNA notebook guideline

**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


## ImageNET 2012 prediction network

The present notebook uses a network trained on the ImageNET 2012 dataset.
The training dataset comprises 1.28 million examples, each associated with a single target class in a list of 1000 possible classes. Due to the dataset size, providing a training notebook compatible with the Free version of Colab is strongly impractical. Still, Python training scripts are provided in the corresponding example directory of CIANNA.

In this notebook, we apply a pre-trained network to the 50000 validation set of ImageNET 2012, and also provide a simplified script to use this network to perform a prediction on an external image. The network architecture is similar to a darknet-19 with a few adjustments to account for the current CIANNA capabilities. The network was first trained at a 224x224 resolution and then further trained at a 448x448 resolution. Network save files are provided for both resolutions. To reduce data download and pre-processing, we provide a test dataset in an already processed binary format containing a subset of 10000 validation images in a squared and centered 500x500 resolution.



In [ ]:
%%shell

cd /content/CIANNA/examples/ImageNET/

python3 - <<EOF

import numpy as np
import albumentations as A
import cv2
import os, glob

#Comment to access system wide install
import sys, glob
sys.path.insert(0,glob.glob('../../src/build/lib.*/')[-1])
import CIANNA as cnn

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

if(not os.path.isdir("ImageNET_aux_data")):
	os.system("wget https://share.obspm.fr/s/Pqbn4cC2azo84Z2/download/ImageNET_aux_data.tar.gz")
	os.system("tar -xvzf ImageNET_aux_data.tar.gz")

if(not os.path.isfile("ImageNET_val10k_bin.tar.gz")):
  os.system("wget https://share.obspm.fr/s/qG9TMDHXWNxr2Qx/download/ImageNET_val10k_bin.tar.gz")
  os.system("tar -xvzf ImageNET_val10k_bin.tar.gz")

processed_data_path = "./val"

image_size_val = 480
image_size = 448
flat_image_slice = image_size*image_size
nb_class = 1000
total_val = 10000
nb_keep_val = 2000
load_epoch = 0

transform_val = A.Compose([
	A.SmallestMaxSize(max_size=image_size_val, interpolation=1, p=1.0),
	A.CenterCrop(width=image_size, height=image_size, p=1.0),
])

val_list = np.loadtxt("ImageNET_aux_data/imagenet_2012_1000classes_val.txt", dtype="str")

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=3, out_dim=nb_class, bias=0.1,
	 b_size=16, comp_meth='C_CUDA', dynamic_load=1, mixed_precision="FP16C_FP32A", adv_size=35)
#Change to FP32 if the allocated GPU does not support FP16 compute

targets_zero = np.zeros((nb_class), dtype="float32")
input_val = np.zeros((nb_keep_val,flat_image_slice*3), dtype="float32")
targets_val = np.zeros((nb_keep_val,nb_class), dtype="float32")

if(load_epoch > 0):
    cnn.load("net_save/net0_s%04d.dat"%load_epoch, load_epoch, bin=1)
else:
  #Not trained as a resolution agnostic network
  if(image_size == 224):
    if(not os.path.isfile("ImageNET_aux_data/net_pretrain_medium_224_acc70.dat")):
      os.system("wget -P ImageNET_aux_data/ https://share.obspm.fr/s/dj69Fm5Gyaenzjw/download/net_pretrain_medium_224_acc70.dat")
    cnn.load("ImageNET_aux_data/net_pretrain_medium_224_acc70.dat", 0, bin=1)
  elif(image_size == 448):
    if(not os.path.isfile("ImageNET_aux_data/net_pretrain_medium_448_acc74.dat")):
      os.system("wget -P ImageNET_aux_data/ https://share.obspm.fr/s/PJaJ6an7amZiQBC/download/net_pretrain_medium_448_acc74.dat")
    cnn.load("ImageNET_aux_data/net_pretrain_medium_448_acc74.dat", 0, bin=1)
  else:
    print("No trained network for the define image resolution")
    exit()


for block in range(int(total_val/nb_keep_val)):
  print("Loading validation dataset block %d"%(block))
  for i in range(0, nb_keep_val):
    patch = np.load(processed_data_path+"/valid_img_%04d.npy"%(i+int(block*nb_keep_val)), allow_pickle=False)

    transformed = transform_val(image=patch)
    patch_aug = transformed['image']

    for depth in range(0,3):
      input_val[i,depth*flat_image_slice:(depth+1)*flat_image_slice] = (patch_aug[:,:,depth].flatten("C") - 100.0)/155.0

    targets_val[i,:] = np.copy(targets_zero[:])
    targets_val[i,int(val_list[i+int(block*nb_keep_val),1])] = 1.0

  cnn.create_dataset("TEST", nb_keep_val, input_val[:,:], targets_val[:,:])

  cnn.forward(repeat=1, no_error=1, saving=2, drop_mode="AVG_MODEL")
  cnn.delete_dataset("TEST")

  os.system("mv fwd_res/net0_%04d.dat fwd_res/net0_%04d_b%d.dat"%(load_epoch, load_epoch, block))

EOF

In [ ]:
#Compute the TOP-1 and TOP-5 errors

%cd /content/CIANNA/examples/ImageNET/

import numpy as np

total_val = 10000
nb_keep_val = 2000
nb_class = 1000
load_epoch = 0

val_list = np.loadtxt("ImageNET_aux_data/imagenet_2012_1000classes_val.txt", dtype="str")

for top_error in [1,5]:

  count = 0
  count_max = 0

  for block in range(int(total_val/nb_keep_val)):
    pred_raw = np.fromfile("fwd_res/net0_%04d_b%d.dat"%(load_epoch, block), dtype="float32")
    predict = np.reshape(pred_raw, (nb_keep_val,nb_class))

    for i in range(0, nb_keep_val):
      ind = np.argpartition(predict[i], -top_error)[-top_error:]
      count_max += np.max(predict[i])

      if(np.isin(int(val_list[i+block*nb_keep_val,1]), ind[:])):
        count += 1

  print ("Top-%d error"%(top_error))
  print (1.0 - count/(total_val))
  print ("Avg max class value")
  print (count_max/(total_val))

## External image prediction

Minimalist example on how to use the network to perform prediction on an external .jpg image.

In [ ]:
%%shell

cd /content/CIANNA/examples/ImageNET/

python3 - <<EOF

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import albumentations as A
import os, cv2
from PIL import Image

#Comment to access system wide install
import sys, glob
sys.path.insert(0,glob.glob('../../src/build/lib.*/')[-1])
import CIANNA as cnn

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

#Minimum deployement setup for prediction on a single image

image_size_val = 480
image_size = 448
flat_image_slice = image_size*image_size
nb_class = 1000

class_list = np.loadtxt("ImageNET_aux_data/imagenet_2012_class_list.txt", dtype="str")[:,1]

if(not os.path.isfile("ImageNET_aux_data/office_1.jpg")):
	os.system("wget -P ImageNET_aux_data/ https://share.obspm.fr/s/GynmcyDtkrsbyLe/download/office_1.jpg")

im = Image.open("ImageNET_aux_data/office_1.jpg", mode='r')

if(im.format != "RGB"):
	im = im.convert('RGB')

patch = np.asarray(im)

transform = A.Compose([
	A.SmallestMaxSize(max_size=image_size_val, interpolation=1, p=1.0),
	A.PadIfNeeded(min_width=image_size_val, min_height=image_size_val, border_mode=cv2.BORDER_CONSTANT, p=1.0),
	A.CenterCrop(width=image_size, height=image_size, p=1.0),
])

transformed = transform(image=patch)
patch_aug = transformed['image']

input_data = f_ar(np.zeros((1,3*image_size*image_size)))
empty_target = f_ar(np.zeros((1,1000)))

for depth in range(0,3):
	input_data[0,depth*flat_image_slice:(depth+1)*flat_image_slice] = (patch_aug[:,:,depth].flatten("C") - 100.0)/155.0

load_epoch = 0
if (len(sys.argv) > 1):
	load_epoch = int(sys.argv[1])

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=3, out_dim=nb_class, bias=0.1,
	 b_size=1, comp_meth='C_CUDA', dynamic_load=1, mixed_precision="FP16C_FP32A", adv_size=35)

cnn.create_dataset("TEST", 1, input_data, empty_target)

if(load_epoch > 0):
	cnn.load("net_save/net0_s%04d.dat"%load_epoch, load_epoch, bin=1)
else:
	#Not trained as a resolution agnostic network
	if(image_size == 224):
		if(not os.path.isfile("ImageNET_aux_data/net_pretrain_medium_224_acc70.dat")):
			os.system("wget -P ImageNET_aux_data/ https://share.obspm.fr/s/dj69Fm5Gyaenzjw/download/net_pretrain_medium_224_acc70.dat")
		cnn.load("ImageNET_aux_data/net_pretrain_medium_224_acc70.dat", 0, bin=1)
	elif(image_size == 448):
		if(not os.path.isfile("ImageNET_aux_data/net_pretrain_medium_448_acc74.dat")):
			os.system("wget -P ImageNET_aux_data/ https://share.obspm.fr/s/PJaJ6an7amZiQBC/download/net_pretrain_medium_448_acc74.dat")
		cnn.load("ImageNET_aux_data/net_pretrain_medium_448_acc74.dat", 0, bin=1)
	else:
		print("No trained network for the define image resolution")
		exit()

cnn.forward(repeat=1, no_error=1, saving=2, drop_mode="AVG_MODEL")

top_error = 5

pred_raw = np.fromfile("fwd_res/net0_%04d.dat"%(load_epoch), dtype="float32")
predict = np.reshape(pred_raw, (1,nb_class))

ind_best = np.argpartition(predict[0], -top_error)[-top_error:]
ind_sort = ind_best[np.argsort(predict[0, ind_best])][::-1]
pred_values = predict[0, ind_sort]

print("Top %d predictions"%(top_error))
for k in range(0, top_error):
  print("%0.2f - %s"%(pred_values[k], class_list[ind_sort[k]]))

fig, ax = plt.subplots(1, 1, dpi=200, constrained_layout=True)

ax.imshow(patch)
ax.axis('off')

for k in range(0, top_error):
	c_text = ax.text(0.02, 1.0-0.04-k*0.04, "%0.2f - %s"%(pred_values[k], class_list[ind_sort[k]]),
		c=plt.cm.tab20(ind_sort[k]%20), fontsize=6, clip_on=True, transform=ax.transAxes)
	c_text.set_path_effects([path_effects.Stroke(linewidth=1.5, foreground='black'), path_effects.Normal()])

plt.savefig("pred_on_image.jpg", dpi=200)

EOF

In [ ]:
#Display the produced JPG
from IPython.display import Image
Image("pred_on_image.jpg", width=960)